# Use the User Interface to start the program after running the .ipynb

# Description: 

This program provides several easily customizable Computer Vision features for images and videos.
Results will be saved to the "OUTPUT" folder.
Multiple features can be simultaneously applied onto a single image or video, or the features can be applied separately and individually. 
Video input can be either livestreamed or a previously saved video.

1. Face Detection

Uses pre-trained ML models to identify faces and classify the emotions and gender of people in an image/video. 
The faces on the image will be marked with rectangular bounding boxes, and the emotions and gender of the faces will be displayed above each box. The total amount of faces detected will be displayed in the top left corner. 
Be aware that this is currently the most computationally expensive of the features. Improvements will be made in the future.

How to use:
Ensure that the "Face detection" button is set to True depending on whether you want to use images or videos.
Drag images to the "INPUT" folder or videos to the "INPUT_VIDEO" folder.




2. Template Matching

Locates a smaller "template image" inside a larger image/video using your choice from a variety of OpenCV methods. The best matches will be marked with bounding boxes.

How to use:
Ensure that the "Template matching" button is set to True depending on whether you want to use images or videos.
Customize constant parameters to choose which OpenCV methods to use.
Drag a single chosen template image to the "TEMPLATE" folder to compare it against each of the images in the "INPUT" folder or for each of the videos in the "INPUT_VIDEO" folder. Template must be smaller than compared images/frames or an Exception will be raised. If there are multiple template images in the "TEMPLATE" folder, only the first will be selected according to alphabetical order.




3. Optical Flow

Draws the optical flow detected in a sequence of images or the frames of a video. 

How to use:
Ensure that the "Optical flow" button is set to True depending on whether you want to use images or videos.
Drag two or more images to the "INPUT" folder or videos to the "INPUT_VIDEO" folder.




4. SIFT (Scale Invariant Feature Transform) 

Compares the SIFT keypoints between two images or between one images and a video. 
Lines will be drawn to show which keypoints match. 

How to use:
Ensure that the "SIFT matching" button is set to True depending on whether you want to use images or videos.
Drag the image to be compared to the "SIFT" folder to compare it against each of the images in the "INPUT" folder or for each of the videos in the "INPUT_VIDEO" folder it. If there are multiple images in the "SIFT" folder, only the first will be selected according to alphabetical order.


# Other considerations

The "Customizable program constants" cell is where all constants that can be modified are found. This is where you can select which features to use or customize the parameters of the features. All constants are well documented with comments.

Multiple images or videos can be processed in a list. 
All images must be .jpg, .jpeg, or .png (but alpha channel not preserved)
All images must be .mp4

 
Example input images will be found in the "data" folder.

The "utils" folder contains the defined functions used in the program.

The "trained_models" folder contains the pretrained face detection models sourced from OpenCV. 

# Credits:
Project programmer:
Ethan Eckmann

Project mentor: 
Xandeep Alexander

Tutorials and code provided by OpenCV and others were crucial in creating this project.

https://github.com/opencv
Face detection, emotion classification, and gender classification machine learning models sourced from OpenCV. 

https://github.com/oarriaga/face_classification
This is where some of the Face Detection code originated. Many modifications were made over the course of the project.

https://docs.opencv.org/4.x/d4/dc6/tutorial_py_template_matching.html
Template Matching tutorial.

https://learnopencv.com/optical-flow-in-opencv/
Optical Flow tutorial.

https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html
SIFT tutorial.

https://github.com/pinecone-io/examples/blob/master/learn/search/image/image-retrieval-ebook/bag-of-visual-words/bag-of-visual-words.ipynb
SIFT bag of visual words / bag of features tutorial.


# Known issues:

1.

Text and bounding boxes may be difficult to see depending on your image/video dimensions. You must customize text and bounding box parameters manually.

2.

Face detection does not work well with a series of images with varying dimsensions and varying distances of faces from the camera. The face detection parameters must be customized for the composition and dimensions of your chosen images. 

3.

The program will produce consistent face detection results for an image, but may produce a different set of consistent results for the same image that has been flipped horizontally (or otherwise edited). Flipping done in the Windows Photo app, any other photo editor, or using cv2.flip() function can change the results. 

Initially having the photo flipped at all in photo editors causes different number of faces to be detected and emotion/gender classification to be different.

Initially having the photo flipped an odd number of times using cv2 functions causes emotion/gender classification to be different only.

Initially having the photo flipped an even number of times using cv2 functions changes nothing.

Differences in emotion, and gender accuracy due to orientation could be caused by the data each respective machine learning model was trained on.

However, the face detection for each image will always be processed in both its orignal state, AND its horizontally flipped state (using cv2) for the most consistent face detection results. The face detection results from both versions of the image are essentially "merged" together before emotion and gender classification. Despite this, the inconsistency problem with face detection still persists. 

I can only speculate that this problem is not caused by any fault of this program but rather caused by metadata changes or differences in .jpg compression techniques between cv2 and various photo editors.

4.

The emotion classifier struggles to identify disgust.

5.

Face Detection with livestream video suffers from stuttering on each position update frame. This is due to the high computational cost of the face detection feature. Other features have stuttering as well, but it is most noticable here. I will work on fixing this flaw in the future. 

6.

Template matching often has low accuracy unless the template is found one-to-one inside an image. This is an inherent weakness of the simplistic template matching algorithim, and template matching was included in this program primarily to showcase the much more robust capabilities of SIFT matching.

7.

Optical flow only tracks the best points initially detected on the first frame of a video. In the future, there will be an option to change this to instead have an update rate.  

8.

SIFT detection struggles when there is heavy noise such as camera blur.

9.

Any form of SIFT detection with videos can sometimes create videos of very large resolution. This may cause the videos to be incompatable with windows media player. To get around this, you can use VLC media player instead.

10.

Optical flow with images will raise an exception if the list of images provided are of varying dimensions.

In [ ]:
# TODO
# Features and improvements that will be implemented in the future:
    # Organize main functions
    # More customization for optical flow, including clear screen intervals 
    # Reduce frame stuttering for livestream
    # Advanced Bag of features using SIFT

# low priority:
    # Better documentation for functions 
    # Change Optical Flow image to work with list of image so that colors are consistent 
    # Make face detection more computationally efficient
    # Make super functions more efficient by removing .copy()s without causing unintentional modifications 
    # Eigenfaces 

In [ ]:
# Imports

import cv2
import os
import sys
from keras.models import load_model
from utils.datasets import get_labels
from utils.detectfaces import face_detection, face_detection_video, face_detection_stream
from utils.templatematch import template_match, template_match_video, template_match_stream
from utils.opticalflow import optical_flow, optical_flow_video, optical_flow_stream
from utils.sift import sift_compare, sift_compare_video, sift_compare_stream
from utils.super import super_image, super_video, super_stream

import ipywidgets as widgets
from ipywidgets import Layout, Output, VBox, Button, ToggleButton
from IPython.display import display
import time
from jupyter_ui_poll import ui_events


# User Interface

In [ ]:
# UI output
output_area = Output()
display(output_area)

In [ ]:
# Customizable program constants

    # -- I/O PARAMETERS --
# String.
# Folder for images to be input for face detection/analysis, template matching, optical flow, and SIFT comparison.
INPUT_IMAGE_FOLDER = 'INPUT'

# String.
# Folder for a single video to be input for face detection/analysis, template matching, optical flow and SIFT comparison.
INPUT_VIDEO_FOLDER = 'INPUT_VIDEO'

# String.
# Folder for a single template image to be input for the purpose of template matching against images or videos.
INPUT_TEMPLATE_FOLDER = 'TEMPLATE'

# String.
# Folder for a single image to be input for the purpose of SIFT comparison against images or videos.
INPUT_SIFT_FOLDER = 'SIFT'

# String.
# Folder for images or videos to be output.
OUTPUT_FOLDER = 'OUTPUT'



    # -- ENABLE/DISABLE PROGRAM FEATURES --
# Boolean.
# If True, performs all enabled features on each image to generate super images, rather than using each of the enabled features separately. 
# Images will be saved to OUTPUT_FOLDER.
# Observe the 'value' property of the toggle button
SUPER_IMAGE = False

# Boolean.
# If True, performs all enabled features on each video to generate a super video, rather than using each of the enabled features separately. 
# Videos will be saved to OUTPUT_FOLDER.
SUPER_VIDEO = False

# Boolean.
# If True, performs face detection and analysis for each image in the INPUT_IMAGE_FOLDER. Draws bounding boxes, identifies the emotion and gender of each face, and counts the number of faces.
# Images will be saved to OUTPUT_FOLDER.
FACEDETECT_IMG_BOOL = False

# Boolean.
# If True, performs template match for each image in INPUT_IMAGE_FOLDER using a single template image in INPUT_TEMPLATE_FOLDER using the cv2 template matching method specified by TEMPLATEMATCH_IMG_METHOD. 
# Images will be saved to OUTPUT_FOLDER.
TEMPLATEMATCH_IMG_BOOL = False

# Boolean.
# If True, performs optical flow using a sequence of images in INPUT_IMAGE_FOLDER. Requires more than one image in INPUT_IMAGE_FOLDER.
# Images will be saved to OUTPUT_FOLDER.
OPFLOW_IMG_BOOL = False

# Boolean.
# If True, performs SIFT (scale invariant feature transform) using a single image in INPUT_SIFT_FOLDER against all images in INPUT_IMAGE_FOLDER.
# Note: The saved images will have higher dimensions then the original INPUT_FOLDER images because the SIFT image is added onto the side of each INPUT_FOLDER image, expanding the width and/or height.
# Images will be saved to OUTPUT_FOLDER.
SIFT_IMG_BOOL = False

# Boolean.
# If True, performs face detection and analysis for each video in INPUT_VIDEO_FOLDER. Draws bounding boxes, identifies the emotion and gender of each face, and counts the number of faces.
# Videos will be saved to OUTPUT_FOLDER.
FACEDETECT_VID_BOOL = False

# Boolean.
# If True, performs template match for each video in INPUT_VIDEO_FOLDER using the single template in INPUT_TEMPLATE_FOLDER using the cv2 template matching method specified by TEMPLATEMATCH_VID_METHOD.
# Videos will be saved to OUTPUT_FOLDER.
TEMPLATEMATCH_VID_BOOL = False

# Boolean.
# If True, performs optical flow for each video in INPUT_VIDEO_FOLDER.
# Videos will be saved to OUTPUT_FOLDER.
OPFLOW_VID_BOOL = False

# Boolean.
# If True, performs SIFT (scale invariant feature transform) using a single image in INPUT_SIFT_FOLDER against all videos in INPUT_VIDEO_FOLDER.
# Note: The saved video will have higher dimensions then your camera resolution because the SIFT image is added onto the side of the video, expanding the width and/or height
# Videos will be saved to OUTPUT_FOLDER.
SIFT_VID_BOOL = False


# Integer.
# How many frames in the video must pass before face detection, template matching, and SIFT data is updated again.
# Higher values lead to worse accuracy but faster processing time. 
# Data is always initially detected on the first frame of the video.
# EX: VID_UPDATE_RATE = 10, means that bounding boxes updated every 10 frames.
VID_UPDATE_RATE = 20

# Boolean.
# If True, use the computer's camera to livestream video feed instead of videos in the INPUT_VIDEO_FOLDER.
# The livestream will appear as a resizeable window 
# Optionally save video.
# The camera is specified using LIVESTREAM_INPUT.
# The camera's settings will be used for determining frame size and frame rate.
# Note: OpenCV is not DPI aware, so windows DPI scaling may cause the window appear too large or too small by default.
# Livestream can be closed by clicking the red X button on the window or by pressing the specified LIVESTREAM_ENDKEY.
LIVESTREAM_BOOL = False

# Integer or URL String.
# 0 for default webcam, or provide another camera index.
# Cameras connected to your system are typically assigned numerical indices starting from 0. Your default webcam is usually index 0. Other connected cameras will have subsequent indices (e.g., 1, 2, etc.).
# If you are trying to access an IP camera or network stream, you would pass the URL of the stream to cv2.VideoCapture() instead of a numerical index.
LIVESTREAM_INPUT = 0

# String.
# If this key on the keyboard is pressed, end the livestream.
LIVESTREAM_ENDKEY = 'q'

# Boolean.
# If True, saves the livestream to OUTPUT_FOLDER after closing.
LIVESTREAM_SAVE_BOOL = False




    # -- COLOR PARAMETERS -- 
# Tuple of integers.
# Example colors in the Blue, Green, Red format used by CV2.
BLUE = (255, 0, 0) 
GREEN = (0, 255, 0) 
RED = (0, 0, 255) 
CYAN = (255, 255, 0)
MAGENTA = (255, 0, 255)
YELLOW = (0, 255, 255)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Color of displayed emotion and gender text.
TEXT_COLOR = RED

# Color of the total faces detected text placed in the corner of the image when using face detection.
TEXT_COLOR_TF = RED

# Color of all text outlines.
OUTLINE_TEXT_COLOR = BLACK

# Color of face detection bounding boxes.
BOX_COLOR = RED

# Color of the template match bounding boxes.
TEMPLATEMATCH_BOX_COLOR = MAGENTA

# Color of the template match method text.
TEMPLATEMATCH_TEXT_COLOR = MAGENTA

# Colors that coordinate with the cv2 template matching methods:
# methods = ['TM_SQDIFF', 'TM_SQDIFF_NORMED', 'TM_CCORR', 'TM_CCORR_NORMED', 'TM_CCOEFF', 'TM_CCOEFF_NORMED']
# Only used if:
# 1. TEMPLATEMATCH_VID_METHODCHOICE is not specified and TEMPLATEMATCH_VID_BOOL = True
# 2. TEMPLATEMATCH_IMG_METHODCHOICE is not specified and SUPER_IMAGE = True
TM_METHODS_COLOR = [BLUE, GREEN, RED, CYAN, MAGENTA, YELLOW]

# Color for lines of optical flow
# If not specified, a list of 100 random colors will be used 
OPFLOW_COLOR = None

# Color for lines that connect SIFT matched keypoints and the color of the matched keypoints.
# If None, colors will be random.
SIFT_LINE_COLOR = GREEN

# Color for points that mark the location of non-matched SIFT keypoints.
# If None, colors will be random.
SIFT_POINT_COLOR = MAGENTA



    # -- TEXT AND BOUNDING BOX PARAMETERS --     
# OUTLINE_TEXT_BOOL - Boolean. If True, all text will be outlined.
# TEXT_SIZE - Float. Font scale of text. When scale > 1, the text is magnified. When 0 < scale < 1, the text is minimized. When scale < 0, the text is mirrored or reversed.
# TEXT_THICK - Integer. Thickness of text.
# BOX_THICK - Integer. Bounding box thickness for face detection and template matching.

# Good starting variables for large sized images:
OUTLINE_TEXT_BOOL = True
TEXT_SIZE = 2
TEXT_THICK = 2
BOX_THICK = 3

# Good starting variables for medium sized images:
# OUTLINE_TEXT_BOOL = True
# TEXT_SIZE = 1
# TEXT_THICK = 2
# BOX_THICK = 2

# Good starting variables for small sized images:
# OUTLINE_TEXT_BOOL = True
# TEXT_SIZE = 0.4
# TEXT_THICK = 1
# BOX_THICK = 1



    # -- FACE DETECTION AND CLASSIFICATION PARAMETERS --
# Face detection parameters:
# scaleFactor - Float. How much the image size is reduced at each image scale. Higher value results in slower processing time but more thorough search
# minNeighbors - Integer. How many neighbors each candidate rectangle should have to retain it. Higher value results in fewer detections
# minSize - Tuple of integers. Minimum possible object size in pixels
# maxSize - Tuple of integers. Maximum possible object size in pixels

    # Good starting variables for medium sized images.
# parameters for face detection (facing forwards):
SCALEFACTOR_F = 1.01 
MINNEIGHBORS_F = 40
MINSIZE_F = (125, 125)
MAXSIZE_F = None
# parameters for face detection (side profile):
SCALEFACTOR_S = 1.01
MINNEIGHBORS_S = 40
MINSIZE_S = (125, 125)
MAXSIZE_S = None

    # Good starting variables for small sized images.
# parameters for face detection (facing forwards):
# SCALEFACTOR_F = 1.1 
# MINNEIGHBORS_F = 10
# MINSIZE_F = (10, 10)
# MAXSIZE_F = None
# # parameters for face detection (side profile):
# SCALEFACTOR_S = 1.1
# MINNEIGHBORS_S = 10
# MINSIZE_S = (10, 10)
# MAXSIZE_S = None


    # alternative, good starting variables for small sized images.
# parameters for face detection (facing forwards):
# SCALEFACTOR_F = 1.05
# MINNEIGHBORS_F = 20
# MINSIZE_F = (10, 10)
# MAXSIZE_F = (200,200)
# # parameters for face detection (side profile):
# SCALEFACTOR_S = 1.05
# MINNEIGHBORS_S = 20
# MINSIZE_S = (10, 10)
# MAXSIZE_S = (200,200)

# Tuple of integers.
# The area for emotion and gender classification expands outwards from the bounding box of a face by an offset.
# These constants do NOT change the bounding box size on the image.
# Recommended to to keep at EMOTION_OFFSETS = (20, 40), GENDER_OFFSETS = (30, 60).
EMOTION_OFFSETS = (20, 40)
GENDER_OFFSETS = (30, 60)


    # -- TEMPLATE MATCHING PARAMETERS --
# List of Strings.
# If any of these conditions are true, all methods in TEMPLATEMATCH_METHODS will be used:
# 1. TEMPLATEMATCH_VID_METHODCHOICE is not specified and TEMPLATEMATCH_VID_BOOL = True
# 2. TEMPLATEMATCH_IMG_METHODCHOICE is not specified and SUPER_IMAGE = True
# The possible CV2 methods for template matching:
# methods = ['TM_SQDIFF', 'TM_SQDIFF_NORMED', 'TM_CCORR', 'TM_CCORR_NORMED', 'TM_CCOEFF', 'TM_CCOEFF_NORMED']
TEMPLATEMATCH_METHODS = ['TM_SQDIFF', 'TM_SQDIFF_NORMED', 'TM_CCORR', 'TM_CCORR_NORMED', 'TM_CCOEFF', 'TM_CCOEFF_NORMED']

# Integer.
# The amount of template matching results to be displayed on the image using a bounding box.
# After the best template match result is found and a bounding box is drawn, the same template image sized area can't be selected twice for the next template match result. This means that overlap usually only occurs when TEMPLATEMATCH_AMOUNT is a very large number.
# Only used for image template match, not video.
# Cannot be lower than 1.
TEMPLATEMATCH_AMOUNT = 1

# String.
# Choice of method for template matching in images.
# If TEMPLATEMATCH_IMG_METHODCHOICE is not found in TEMPLATEMATCH_METHODS: all methods in TEMPLATEMATCH_METHODS will be considered for each image in INPUT. If SUPER_IMAGE = True, all methods will be simultaneously applied onto each of the input images. Otherwise, each method will be applied individually for each input image, which can lead to a large amount of resulting images to be created.  
TEMPLATEMATCH_IMG_METHODCHOICE = 'TM_SQDIFF_NORMED'

# String.
# Choice of method for template matching in videos.
# If TEMPLATEMATCH_VID_METHODCHOICE is not found in TEMPLATEMATCH_METHODS: all methods in TEMPLATEMATCH_METHODS will be simultaneously applied for each video.
TEMPLATEMATCH_VID_METHODCHOICE = None



    # -- OPTICAL FLOW PARAMETERS --
# Integer.
# How many random colors should be generated in a list if optical flow features are enabled but OPFLOW_COLOR is not specified. Note that colors can be repeated.
# Each random color will correlate with an optical flow line and point. 
# If the list of random colors is smaller than the number of detected optical flow points then the program will raise an indexOutOfBounds error, so it is important that this number is at least 1000 to be safe. 
# If not specified, defaults to 1000.
OPFLOW_TOTAL_RANDOM_COLORS = 1000



    # -- SIFT PARAMETERS --
# Float.
# Used to discard ambiguous SIFT matches.
# Higher values lead to more SIFT matches detected, lower values lead to less SIFT matches detected.
# Explanation:
# Each keypoint of the first image is matched with a number of keypoints from the second image. We keep the 2 best matches for each keypoint (best matches = the ones with the smallest distance measurement). Lowe's test checks that the two distances are sufficiently different. If they are not, then the keypoint is determined to be ambiguous and is eliminated. 
    # Simple example:
    # Initally, the two best matches are 
    # m.distance = 309, n.distance = 329
    # the ratio_threshold = 0.20 and when multiplied with n.distance = 65

    # logic:
        # if m.distance < ratio_threshold*n.distance:
        #   m is a good match
        # else:
        #   m is a bad match

    # appling the logic
        # 309 < 65:
        # result: m is a bad match
        # lower ratio_threshold values causes less matches.
SIFT_RATIO_THRESHOLD = 0.65

# Enumeration (Integer).
# Three options:
# cv2.DrawMatchesFlags_DEFAULT (0) - Mark all SIFT features in the images using circles with color = SIFT_POINT_COLOR and draw lines with color = SIFT_LINE_COLOR to connect matches.
# cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS (2) - Draw lines with color = SIFT_LINE_COLOR to connect matches only.
# cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS (4) - Mark all SIFT features with keypoint size and orientation in the images using circles with color = SIFT_POINT_COLOR and draw lines with color = SIFT_LINE_COLOR to connect matches.
# If you do not want any lines on the image, you must set SIFT_LINE_COLOR = None.
# Combine multiple flags with with " | "
# Ex: SIFT_FLAG = cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS | cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS
# Note: The cv2.DrawMatchesFlags_DRAW_OVER_OUTIMG (1) flag is currently not implemented in this program due to the fact that it would be difficult to implement and would conflict with super videos or super images. Trying to use it will cause an exception to be raised. An example of how this flag would work normally is included in the data/examples folder for viewing.
# Note:
# It technically is possible to remove the lines, but because the lines include matched keypoints, removing the lines and also keeping MATCHED keypoints is impossible using the cv2.drawMatchesKnn function. Workarounds are possible, but one was not implemented for this program.
SIFT_FLAG = cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS

# Boolean.
# If SIFT_SIDE is True, then the SIFT image will appear to the right of the image/video it is compared to 
# If SIFT_SIDE is False, then the SIFT image will appear to the left of the imag/video it is compared to 
SIFT_RSIDE = True

In [ ]:
# UI for important boolean values and starting program
# Program waits until buttons pressed
# Most of UI code AI generated for convenience 

# Dictionary to hold the ToggleButton widgets for easier management
button_widgets = {}

# Dictionary to map variable names (strings) to the actual global boolean variables
# This helps in referencing them in the observer function
global_boolean_vars = {
    'Super Image': lambda: SUPER_IMAGE, 
    'Super Video': lambda: SUPER_VIDEO,
    'Livestream': lambda: LIVESTREAM_BOOL,
    'Save livestream': lambda: LIVESTREAM_SAVE_BOOL,
    'Face detection for images': lambda: FACEDETECT_IMG_BOOL,
    'Template matching for images': lambda: TEMPLATEMATCH_IMG_BOOL,
    'Optical flow for images': lambda: OPFLOW_IMG_BOOL,
    'SIFT matching for images': lambda: SIFT_IMG_BOOL,
    'Face detection for videos': lambda: FACEDETECT_VID_BOOL,
    'Template matching for videos': lambda: TEMPLATEMATCH_VID_BOOL,
    'Optical flow for videos': lambda: OPFLOW_VID_BOOL,
    'SIFT matching for videos': lambda: SIFT_VID_BOOL,
}

# Define the observer function (updated to handle global variables)
def on_toggle_change(change, var_name):
    global SUPER_IMAGE, SUPER_VIDEO, LIVESTREAM_BOOL, LIVESTREAM_SAVE_BOOL, FACEDETECT_IMG_BOOL, TEMPLATEMATCH_IMG_BOOL, OPFLOW_IMG_BOOL, SIFT_IMG_BOOL, FACEDETECT_VID_BOOL, TEMPLATEMATCH_VID_BOOL, OPFLOW_VID_BOOL, SIFT_VID_BOOL # Declare global to modify

    # Update the corresponding global boolean variable based on var_name
    if var_name == 'Super Image':
        SUPER_IMAGE = change.new
    elif var_name == 'Super Video':
        SUPER_VIDEO = change.new
    elif var_name == 'Livestream':
        LIVESTREAM_BOOL = change.new
    elif var_name == 'Save livestream':
        LIVESTREAM_SAVE_BOOL = change.new
    elif var_name == 'Face detection for images':
        FACEDETECT_IMG_BOOL = change.new
    elif var_name == 'Template matching for images':
        TEMPLATEMATCH_IMG_BOOL = change.new
    elif var_name == 'Optical flow for images':
        OPFLOW_IMG_BOOL = change.new
    elif var_name == 'SIFT matching for images':
        SIFT_IMG_BOOL = change.new
    elif var_name == 'Face detection for videos':
        FACEDETECT_VID_BOOL = change.new
    elif var_name == 'Template matching for videos':
        TEMPLATEMATCH_VID_BOOL = change.new
    elif var_name == 'Optical flow for videos':
        OPFLOW_VID_BOOL = change.new
    elif var_name == 'SIFT matching for videos':
        SIFT_VID_BOOL = change.new

    # Update the output area to show the change
    with output_area:
        # Retrieve the updated value from the global variable
        updated_value = change.new # This directly reflects the button's new state  

    # Update the button's icon and style based on its new state
    button = button_widgets[var_name]
    if change.new:
        button.icon = 'check'
        button.button_style = 'primary'
    else:
        button.icon = ''
        button.button_style = 'danger'

# Create buttons and attach observers
for name, get_initial_value_func in global_boolean_vars.items():
    initial_value = get_initial_value_func()
    button_style = 'primary' if initial_value else 'danger'

    button = ToggleButton(
        value=initial_value,
        description=name,
        button_style=button_style,
        tooltip=f'Toggle {name}',
        icon='check' if initial_value else '',
        layout=Layout(width='200px', height='50px')
    )
    button_widgets[name] = button

    # Attach the observer, passing the variable name using a lambda for closure
    button.observe(lambda change, var_name_for_closure=name: on_toggle_change(change, var_name_for_closure), names='value')

# Arrange the buttons in a vertical box
buttons_box = VBox(list(button_widgets.values()))

with output_area: # Display the widgets
    print("Interact with the buttons below:")




# Boolean to start and stop program
is_running = False
# Button to start program
def start_program(b):
    # Callback function for the start button
    global is_running
    if not is_running:
        is_running = True
        # Start main long_running_task in a separate thread
        # This is crucial so the Jupyter kernel doesn't freeze
        # while the task is running.
        # thread = threading.Thread(target=long_running_task(output_area))
        # thread.start()
        start_button.disabled = True
        with output_area:
            print("Starting program...")

# Create the buttons
start_button = Button(description="Start Program", button_style='success')

# Register the callback functions
start_button.on_click(start_program)

with output_area:
    display(buttons_box, start_button)

with ui_events() as poll:
    while not is_running:
        poll(10)  # Poll for UI events, including button clicks
        time.sleep(0.1)  # Briefly sleep to avoid busy-waiting





In [ ]:
# Setup
# paths for the pre-trained models 
detection_model_front_path = 'trained_models/detection_models/haarcascade_frontalface_default.xml'
detection_model_side_path = 'trained_models/detection_models/haarcascade_profileface.xml'
emotion_model_path = 'trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
gender_model_path = 'trained_models/gender_models/gender_mini_XCEPTION.21-0.95.hdf5'

# loading models
# face detection and emotion/gender models need to be loaded differently
FACEDETECTION_FRONT = cv2.CascadeClassifier(detection_model_front_path)
FACEDETECTION_SIDE = cv2.CascadeClassifier(detection_model_side_path)
EMOTION_CLASSIFIER = load_model(emotion_model_path, compile=False)
GENDER_CLASSIFIER = load_model(gender_model_path, compile=False)

# labels for the fer2013 dataset that emotion model is using
# emotion_labels: {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'sad', 5: 'surprise', 6: 'neutral'}
EMOTION_LABELS = get_labels('fer2013')

# labels for the imdb dataset that gender model is using 
# gender_labels: {0: 'woman', 1: 'man'}
GENDER_LABELS = get_labels('imdb')

In [ ]:
# Loading images

image_list = []
video_paths_list = []
image_extensions = ('.jpg', '.jpeg', '.png') 
video_extensions = ('.mp4')
# sorts numerically for numbers and alphabetically (Unicode comparison) for strings, in ascending order
filenames_images = sorted([f for f in os.listdir(INPUT_IMAGE_FOLDER) if f.lower().endswith(image_extensions)]) 
filenames_video = sorted([f for f in os.listdir(INPUT_VIDEO_FOLDER) if f.lower().endswith(video_extensions)]) 
filenames_templates = sorted([f for f in os.listdir(INPUT_TEMPLATE_FOLDER) if f.lower().endswith(image_extensions)]) 
filenames_sift = sorted([f for f in os.listdir(INPUT_SIFT_FOLDER) if f.lower().endswith(image_extensions)]) 

# Error handling
if (len(filenames_images) == 0 or filenames_images is None) and (FACEDETECT_IMG_BOOL or TEMPLATEMATCH_IMG_BOOL or OPFLOW_IMG_BOOL or SIFT_IMG_BOOL):
    sys.exit(f'Error: No images were input. Place image(s) in the {INPUT_IMAGE_FOLDER} folder.') 

if LIVESTREAM_BOOL == False and (len(filenames_video) == 0 or filenames_video is None) and (FACEDETECT_VID_BOOL or TEMPLATEMATCH_VID_BOOL or OPFLOW_VID_BOOL or SIFT_VID_BOOL):
    sys.exit(f'Error: No videos were input. Place video(s) in the {INPUT_VIDEO_FOLDER} folder.') 

if LIVESTREAM_BOOL and (FACEDETECT_VID_BOOL == False and TEMPLATEMATCH_VID_BOOL == False and OPFLOW_VID_BOOL == False and SIFT_VID_BOOL == False):
    sys.exit(f'Error: Livestream enabled but no video features enabled.') 

if LIVESTREAM_BOOL:
    c = cv2.VideoCapture(LIVESTREAM_INPUT)
    if not c.isOpened():
        print(f"Error: Could not open camera with index {LIVESTREAM_INPUT}")

if (len(filenames_templates) == 0 or filenames_templates is None) and (TEMPLATEMATCH_IMG_BOOL or TEMPLATEMATCH_VID_BOOL):
    sys.exit(f'Error: Template match for images is enabled, but no templates were provided. Place a template image in the {INPUT_TEMPLATE_FOLDER} folder.') 

if (len(filenames_images) < 2 or filenames_images is None) and OPFLOW_IMG_BOOL:
    sys.exit(f'Error: Optical flow for images is not possible without two or more images. Place image(s) in the {INPUT_IMAGE_FOLDER} folder.') 

if (len(filenames_sift) == 0 or filenames_sift is None) and (SIFT_IMG_BOOL or SIFT_VID_BOOL):
    sys.exit(f'Error: SIFT match for images is enabled, but the SIFT image to be compared was not provided. Place a image in the {INPUT_SIFT_FOLDER} folder.') 


image_list = []
# if image is enabled, create list of read images to be used in program 
if FACEDETECT_IMG_BOOL or TEMPLATEMATCH_IMG_BOOL or OPFLOW_IMG_BOOL or SIFT_IMG_BOOL:
    for f in filenames_images:
        full_path = os.path.join(INPUT_IMAGE_FOLDER, f)
        image = cv2.imread(full_path, cv2.IMREAD_COLOR)
        image_list.append(image)

video_paths_list = []
# if video is enabled, create list of video paths
if FACEDETECT_VID_BOOL or TEMPLATEMATCH_VID_BOOL or OPFLOW_VID_BOOL or SIFT_VID_BOOL:
    for f in filenames_video:
        full_path = os.path.join(INPUT_VIDEO_FOLDER, f)
        video_paths_list.append(full_path)

# if template matching is enabled, select only the first template image from TEMPLATE
template = None
if TEMPLATEMATCH_IMG_BOOL or TEMPLATEMATCH_VID_BOOL:
    full_path = os.path.join(INPUT_TEMPLATE_FOLDER, filenames_templates[0])
    template = cv2.imread(full_path, cv2.IMREAD_COLOR)

# if SIFT is enabled, select only the first image from SIFT
sift_image = None
if SIFT_IMG_BOOL or SIFT_VID_BOOL:
    full_path = os.path.join(INPUT_SIFT_FOLDER, filenames_sift[0])
    sift_image = cv2.imread(full_path, cv2.IMREAD_COLOR)

In [ ]:
# Function calls

# Perform image features separately


if not SUPER_IMAGE:
    # Optional: Face detection, emotion and gender classification for each image in INPUT
    if FACEDETECT_IMG_BOOL:
        for i in range(len(image_list)):
            filename = f'facedetect_{i+1}'
            full_output_path = os.path.join(OUTPUT_FOLDER, filename)
            path, img = face_detection(image_list[i], FACEDETECTION_FRONT, SCALEFACTOR_F, MINNEIGHBORS_F, MINSIZE_F, MAXSIZE_F, FACEDETECTION_SIDE, SCALEFACTOR_S, MINNEIGHBORS_S, MINSIZE_S, MAXSIZE_S, EMOTION_CLASSIFIER, GENDER_CLASSIFIER, EMOTION_OFFSETS, GENDER_OFFSETS, EMOTION_LABELS, GENDER_LABELS, BOX_COLOR, BOX_THICK, TEXT_COLOR, TEXT_SIZE, TEXT_THICK, TEXT_COLOR_TF, OUTLINE_TEXT_BOOL, OUTLINE_TEXT_COLOR, full_output_path)
            cv2.imwrite(path, img)

    # Optional: Template match compare using the single template image in TEMPLATE against each image in INPUT 
    if TEMPLATEMATCH_IMG_BOOL:
        paths_list = []
        img_list = []
        for i in range(len(image_list)):
            filename = f'templatematch_{i+1}'
            full_output_path = os.path.join(OUTPUT_FOLDER, filename)
            temp_paths_list, temp_img_list = template_match(image_list[i], template, TEMPLATEMATCH_METHODS, TEMPLATEMATCH_BOX_COLOR, TEMPLATEMATCH_TEXT_COLOR, TEXT_SIZE, TEXT_THICK, BOX_THICK, OUTLINE_TEXT_BOOL, OUTLINE_TEXT_COLOR, full_output_path, TEMPLATEMATCH_IMG_METHODCHOICE, TEMPLATEMATCH_AMOUNT)
            paths_list = paths_list + temp_paths_list
            img_list = img_list + temp_img_list
        for j in range(len(paths_list)):
            cv2.imwrite(paths_list[j], img_list[j])

    # Optional: Optical flow for the sequence of images in INPUT
    if OPFLOW_IMG_BOOL:
        for i in range(len(image_list) -1):
            filename = f'opflow_{i+1}-{i+2}'
            full_output_path = os.path.join(OUTPUT_FOLDER, filename)
            path, img = optical_flow(image_list[i], image_list[i+1], full_output_path, total_random_colors=OPFLOW_TOTAL_RANDOM_COLORS, color=OPFLOW_COLOR)
            cv2.imwrite(path, img)

    # Optional: Compare SIFT features of one image against all other images 
    if SIFT_IMG_BOOL:
        for i in range(len(image_list)): # start index at 1 to skip the first element 
            filename = f'SIFT_{i+1}'
            full_output_path = os.path.join(OUTPUT_FOLDER, filename)
            path, img = sift_compare(image_list[i], sift_image, SIFT_RATIO_THRESHOLD, SIFT_LINE_COLOR, SIFT_POINT_COLOR, SIFT_RSIDE, full_output_path, SIFT_FLAG)
            cv2.imwrite(path, img)


# SUPER IMAGE
# Perform all image features together on each image
# The image markups from one feature will not have any effect on the results of another feature
if SUPER_IMAGE:  
    filename = 'super'
    full_output_path = os.path.join(OUTPUT_FOLDER, filename)
    super_image(image_list, FACEDETECTION_FRONT, SCALEFACTOR_F, MINNEIGHBORS_F, MINSIZE_F, MAXSIZE_F, FACEDETECTION_SIDE, SCALEFACTOR_S, MINNEIGHBORS_S, MINSIZE_S, MAXSIZE_S, EMOTION_CLASSIFIER, GENDER_CLASSIFIER, EMOTION_OFFSETS, GENDER_OFFSETS, EMOTION_LABELS, GENDER_LABELS, template, TEMPLATEMATCH_METHODS, TM_METHODS_COLOR, TEMPLATEMATCH_IMG_METHODCHOICE, TEMPLATEMATCH_AMOUNT, sift_image, SIFT_RATIO_THRESHOLD, SIFT_LINE_COLOR, SIFT_POINT_COLOR, SIFT_RSIDE, SIFT_FLAG, BOX_COLOR, BOX_THICK, TEXT_COLOR, TEXT_SIZE, TEXT_THICK, TEXT_COLOR_TF, TEMPLATEMATCH_BOX_COLOR, TEMPLATEMATCH_TEXT_COLOR, OUTLINE_TEXT_BOOL, OUTLINE_TEXT_COLOR, full_output_path, FACEDETECT_IMG_BOOL, TEMPLATEMATCH_IMG_BOOL, OPFLOW_IMG_BOOL, SIFT_IMG_BOOL,OPFLOW_TOTAL_RANDOM_COLORS, OPFLOW_COLOR)


# Perform video features separately
if not SUPER_VIDEO:
    # Optional: Face detection, emotion and gender classification using each video in INPUT_VIDEO or with livestream
    if FACEDETECT_VID_BOOL:
        if LIVESTREAM_BOOL:
            filename = f'facedetect_stream'
            full_output_path = os.path.join(OUTPUT_FOLDER, filename)
            face_detection_stream(LIVESTREAM_INPUT, LIVESTREAM_ENDKEY, LIVESTREAM_SAVE_BOOL, VID_UPDATE_RATE, FACEDETECTION_FRONT, SCALEFACTOR_F, MINNEIGHBORS_F, MINSIZE_F, MAXSIZE_F, FACEDETECTION_SIDE, SCALEFACTOR_S, MINNEIGHBORS_S, MINSIZE_S, MAXSIZE_S, EMOTION_CLASSIFIER, GENDER_CLASSIFIER, EMOTION_OFFSETS, GENDER_OFFSETS, EMOTION_LABELS, GENDER_LABELS, BOX_COLOR, BOX_THICK, TEXT_COLOR, TEXT_SIZE, TEXT_THICK, TEXT_COLOR_TF, OUTLINE_TEXT_BOOL, OUTLINE_TEXT_COLOR, full_output_path)
        else:
            for i in range(len(video_paths_list)):
                filename = f'facedetect_vid_{i+1}'
                full_output_path = os.path.join(OUTPUT_FOLDER, filename)
                face_detection_video(video_paths_list[i], VID_UPDATE_RATE, FACEDETECTION_FRONT, SCALEFACTOR_F, MINNEIGHBORS_F, MINSIZE_F, MAXSIZE_F, FACEDETECTION_SIDE, SCALEFACTOR_S, MINNEIGHBORS_S, MINSIZE_S, MAXSIZE_S, EMOTION_CLASSIFIER, GENDER_CLASSIFIER, EMOTION_OFFSETS, GENDER_OFFSETS, EMOTION_LABELS, GENDER_LABELS, BOX_COLOR, BOX_THICK, TEXT_COLOR, TEXT_SIZE, TEXT_THICK, TEXT_COLOR_TF, OUTLINE_TEXT_BOOL, OUTLINE_TEXT_COLOR, full_output_path)

    # Optional: Template match compare using the single template image in TEMPLATE against each video in INPUT_VIDEO or with livestream
    if TEMPLATEMATCH_VID_BOOL:
        if LIVESTREAM_BOOL:
            filename = f'templatematch_stream'
            full_output_path = os.path.join(OUTPUT_FOLDER, filename)
            template_match_stream(LIVESTREAM_INPUT, LIVESTREAM_ENDKEY, LIVESTREAM_SAVE_BOOL, VID_UPDATE_RATE, template, TEMPLATEMATCH_METHODS, TEMPLATEMATCH_BOX_COLOR, TEMPLATEMATCH_TEXT_COLOR, TEXT_SIZE, TEXT_THICK, BOX_THICK, OUTLINE_TEXT_BOOL, OUTLINE_TEXT_COLOR, full_output_path, TM_METHODS_COLOR, TEMPLATEMATCH_VID_METHODCHOICE)
        else:
            for i in range(len(video_paths_list)):
                filename = f'templatematch_vid_{i+1}'
                full_output_path = os.path.join(OUTPUT_FOLDER, filename)
                template_match_video(video_paths_list[i], VID_UPDATE_RATE, template, TEMPLATEMATCH_METHODS, TEMPLATEMATCH_BOX_COLOR, TEMPLATEMATCH_TEXT_COLOR, TEXT_SIZE, TEXT_THICK, BOX_THICK, OUTLINE_TEXT_BOOL, OUTLINE_TEXT_COLOR, full_output_path, TM_METHODS_COLOR, TEMPLATEMATCH_VID_METHODCHOICE)

    # Optional: Optical flow using each video in INPUT_VIDEO
    if OPFLOW_VID_BOOL:
        if LIVESTREAM_BOOL:
            filename = f'opflow_stream'
            full_output_path = os.path.join(OUTPUT_FOLDER, filename)
            optical_flow_stream(LIVESTREAM_INPUT, LIVESTREAM_ENDKEY, LIVESTREAM_SAVE_BOOL, full_output_path, OPFLOW_TOTAL_RANDOM_COLORS, OPFLOW_COLOR)
        else:
            for i in range(len(video_paths_list)):
                filename = f'opflow_vid_{i+1}'
                full_output_path = os.path.join(OUTPUT_FOLDER, filename)
                optical_flow_video(video_paths_list[i], full_output_path, OPFLOW_TOTAL_RANDOM_COLORS, OPFLOW_COLOR)

    # Optional: Compare SIFT features of one image against all videos 
    if SIFT_VID_BOOL:
        if LIVESTREAM_BOOL:
            filename = f'SIFT_stream'
            full_output_path = os.path.join(OUTPUT_FOLDER, filename)
            sift_compare_stream(LIVESTREAM_INPUT, LIVESTREAM_ENDKEY, LIVESTREAM_SAVE_BOOL, VID_UPDATE_RATE, sift_image, SIFT_RATIO_THRESHOLD, SIFT_LINE_COLOR, SIFT_POINT_COLOR, SIFT_RSIDE, full_output_path, SIFT_FLAG)
        else:
            for i in range(len(video_paths_list)):
                filename = f'SIFT_vid_{i+1}'
                full_output_path = os.path.join(OUTPUT_FOLDER, filename)
                sift_compare_video(video_paths_list[i], VID_UPDATE_RATE, sift_image, SIFT_RATIO_THRESHOLD, SIFT_LINE_COLOR, SIFT_POINT_COLOR, SIFT_RSIDE, full_output_path, SIFT_FLAG)


# SUPER VIDEO
# Perform all video features together on each video
# The frame markups from one feature will not have any effect on the results of another feature
if SUPER_VIDEO:
    if LIVESTREAM_BOOL:
        filename = f'super_stream'
        full_output_path = os.path.join(OUTPUT_FOLDER, filename)
        super_stream(LIVESTREAM_INPUT, LIVESTREAM_ENDKEY, LIVESTREAM_SAVE_BOOL, VID_UPDATE_RATE, FACEDETECTION_FRONT, SCALEFACTOR_F, MINNEIGHBORS_F, MINSIZE_F, MAXSIZE_F, FACEDETECTION_SIDE, SCALEFACTOR_S, MINNEIGHBORS_S, MINSIZE_S, MAXSIZE_S, EMOTION_CLASSIFIER, GENDER_CLASSIFIER, EMOTION_OFFSETS, GENDER_OFFSETS, EMOTION_LABELS, GENDER_LABELS, template, TEMPLATEMATCH_METHODS, TM_METHODS_COLOR, TEMPLATEMATCH_VID_METHODCHOICE, sift_image, SIFT_RATIO_THRESHOLD, SIFT_LINE_COLOR, SIFT_POINT_COLOR, SIFT_RSIDE, SIFT_FLAG, BOX_COLOR, BOX_THICK, TEXT_COLOR, TEXT_SIZE, TEXT_THICK, TEXT_COLOR_TF, TEMPLATEMATCH_BOX_COLOR, TEMPLATEMATCH_TEXT_COLOR, OUTLINE_TEXT_BOOL, OUTLINE_TEXT_COLOR, full_output_path, FACEDETECT_VID_BOOL, TEMPLATEMATCH_VID_BOOL, OPFLOW_VID_BOOL, SIFT_VID_BOOL, OPFLOW_TOTAL_RANDOM_COLORS, OPFLOW_COLOR)
    else:
        for i in range(len(video_paths_list)):
            filename = f'super_{i+1}'
            full_output_path = os.path.join(OUTPUT_FOLDER, filename)
            super_video(video_paths_list[i], VID_UPDATE_RATE, FACEDETECTION_FRONT, SCALEFACTOR_F, MINNEIGHBORS_F, MINSIZE_F, MAXSIZE_F, FACEDETECTION_SIDE, SCALEFACTOR_S, MINNEIGHBORS_S, MINSIZE_S, MAXSIZE_S, EMOTION_CLASSIFIER, GENDER_CLASSIFIER, EMOTION_OFFSETS, GENDER_OFFSETS, EMOTION_LABELS, GENDER_LABELS, template, TEMPLATEMATCH_METHODS, TM_METHODS_COLOR, TEMPLATEMATCH_VID_METHODCHOICE, sift_image, SIFT_RATIO_THRESHOLD, SIFT_LINE_COLOR, SIFT_POINT_COLOR, SIFT_RSIDE, SIFT_FLAG, BOX_COLOR, BOX_THICK, TEXT_COLOR, TEXT_SIZE, TEXT_THICK, TEXT_COLOR_TF, TEMPLATEMATCH_BOX_COLOR, TEMPLATEMATCH_TEXT_COLOR, OUTLINE_TEXT_BOOL, OUTLINE_TEXT_COLOR, full_output_path, FACEDETECT_VID_BOOL, TEMPLATEMATCH_VID_BOOL, OPFLOW_VID_BOOL, SIFT_VID_BOOL, OPFLOW_TOTAL_RANDOM_COLORS, OPFLOW_COLOR)

